In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import hstack, csr_matrix
import math

# IMDB and Movies merge

In [2]:
imdb = pd.read_csv("imdb_top_1000.csv")

In [3]:
imdb.shape

(1000, 16)

In [4]:
imdb.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [5]:
imdb.rename(columns = {'Series_Title':'title'}, inplace = True)

In [6]:
imdb.head()

,Poster_Link,title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [7]:
imdb = imdb[["title","Overview"]]

In [8]:
imdb.head()

,title,Overview
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...
1,The Godfather,An organized crime dynasty's aging patriarch t...
2,The Dark Knight,When the menace known as the Joker wreaks havo...
3,The Godfather: Part II,The early life and career of Vito Corleone in ...
4,12 Angry Men,A jury holdout attempts to prevent a miscarria...


In [9]:
moviess = pd.read_csv("movie.csv")

In [10]:
moviess.shape

(27278, 3)

In [11]:
moviess.title = moviess.title.str[:-7]

In [12]:
moviess.head()

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [13]:
moviess = moviess.drop_duplicates(subset=['title'])

In [14]:
moviess.shape

(26216, 3)

In [15]:
Merged_movie_imdb = pd.merge(moviess,imdb,on="title")

In [16]:
Merged_movie_imdb.shape

(482, 4)

In [17]:
Merged_movie_imdb.head()

,movieId,title,genres,Overview
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
1,6,Heat,Action|Crime|Thriller,A group of professional bank robbers start to ...
2,7,Sabrina,Comedy|Romance,A playboy becomes interested in the daughter o...
3,16,Casino,Crime|Drama,"A tale of greed, deception, money, power, and ..."
4,17,Sense and Sensibility,Drama|Romance,"Rich Mr. Dashwood dies, leaving his second wif..."


# Merge with Ratings now

In [18]:
ratings = pd.read_csv("rating.csv")

In [19]:
ratings.shape

(20000263, 4)

In [20]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [21]:
#ratings = ratings.drop_duplicates(subset=["userId",'movieId'], keep='last')

In [22]:
ratings.shape

(20000263, 4)

In [23]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


# Merge now

In [24]:
Final = pd.merge(ratings,Merged_movie_imdb,on="movieId")

In [25]:
Final.head()

,userId,movieId,rating,timestamp,title,genres,Overview
0,1,223,4.0,2005-04-02 23:46:13,Clerks,Comedy,A day in the lives of two convenience clerks n...
1,3,223,5.0,1999-12-11 13:20:44,Clerks,Comedy,A day in the lives of two convenience clerks n...
2,16,223,3.0,2001-05-27 13:26:17,Clerks,Comedy,A day in the lives of two convenience clerks n...
3,21,223,3.0,2001-06-10 16:24:31,Clerks,Comedy,A day in the lives of two convenience clerks n...
4,23,223,5.0,1998-12-24 00:05:12,Clerks,Comedy,A day in the lives of two convenience clerks n...


In [26]:
Final.shape

(4437944, 7)

In [27]:
Final1 = Final.drop_duplicates(subset=["userId",'movieId'], keep='last')

In [28]:
Final1.shape

(4437943, 7)

In [29]:
Final1.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
Overview     0
dtype: int64

In [30]:
Final1.sort_values(by=['userId'])

,userId,movieId,rating,timestamp,title,genres,Overview
0,1,223,4.0,2005-04-02 23:46:13,Clerks,Comedy,A day in the lives of two convenience clerks n...
795020,1,2959,4.0,2004-09-10 03:08:18,Fight Club,Action|Crime|Drama|Thriller,An insomniac office worker and a devil-may-car...
835126,1,3499,4.0,2005-04-02 23:35:18,Misery,Drama|Horror|Thriller,After a famous author is rescued from a car cr...
844262,1,4011,4.0,2005-04-02 23:43:26,Snatch,Comedy|Crime|Thriller,"Unscrupulous boxing promoters, violent bookmak..."
861879,1,4027,4.0,2005-04-02 23:44:20,"O Brother, Where Art Thou?",Adventure|Comedy|Crime,"In the deep south during the 1930s, three esca..."
...,...,...,...,...,...,...,...
3218717,138493,2997,4.5,2009-10-17 19:26:24,Being John Malkovich,Comedy|Drama|Fantasy,A puppeteer discovers a portal that leads lite...
3718485,138493,68954,4.5,2009-11-13 15:42:00,Up,Adventure|Animation|Children|Drama,78-year-old Carl Fredricksen travels to Paradi...
2840558,138493,6711,5.0,2009-10-17 22:13:56,Lost in Translation,Comedy|Drama|Romance,A faded movie star and a neglected young woman...
3730768,138493,70286,5.0,2009-11-13 15:42:24,District 9,Mystery|Sci-Fi|Thriller,Violence ensues after an extraterrestrial race...


# So, Final1 is the data on which you can apply everything.!

In [31]:
#Final1.to_csv("Product data.csv")

# Check the sparsity of the data

In [32]:
n_users = Final1['userId'].nunique()
n_movies = Final1['movieId'].nunique()

print('Number of users:', n_users)
print('Number of movies:', n_movies)

Number of users: 138051
Number of movies: 479


In [33]:
# Check data sparsity

sparsity = round(1.0 - Final1.shape[0] / float(n_users * n_movies), 3)
print('The sparsity level of data is ' +  str(sparsity * 100) + '%')

The sparsity level of data is 93.30000000000001%


# Lets apply collaborative filtering now.!

In [34]:
user_item_m = Final1.pivot('userId','movieId','rating').fillna(0)
print(f'Shape: {user_item_m.shape}')

Shape: (138051, 479)


In [35]:
user_item_m.iloc[:10,:15].astype('i1').T.join(moviess.set_index('movieId').title).set_index('title').T.rename_axis('userId')

title,Toy Story,Heat,Sabrina,Casino,Sense and Sensibility,Braveheart,Taxi Driver,Apollo 13,Die Hard: With a Vengeance,Before Sunrise,Clerks,Ed Wood,Little Women,Miracle on 34th Street,Once Were Warriors
userId,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0
2,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,5,0,0,0,0
4,0,3,0,0,0,0,0,0,3,0,0,0,0,0,0
5,0,0,0,0,3,4,0,5,0,0,0,3,0,0,0
6,5,0,5,0,5,0,0,0,0,0,0,0,0,0,0
7,0,0,3,3,2,0,0,0,0,0,0,0,0,0,0
8,4,3,0,0,0,5,0,4,3,0,0,0,0,4,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
from sklearn.metrics.pairwise import cosine_similarity

# User-Based CF

In [37]:
user_item_m = user_item_m.iloc[:7000, : ]

In [38]:
X_user = cosine_similarity(user_item_m)

In [39]:
X_user.shape

(7000, 7000)

In [40]:
X_user[:10,:8].round(3)

array([[1.   , 0.273, 0.446, 0.055, 0.221, 0.   , 0.184, 0.086],
       [0.273, 1.   , 0.405, 0.228, 0.311, 0.   , 0.365, 0.217],
       [0.446, 0.405, 1.   , 0.114, 0.194, 0.081, 0.324, 0.104],
       [0.055, 0.228, 0.114, 1.   , 0.181, 0.   , 0.21 , 0.444],
       [0.221, 0.311, 0.194, 0.181, 1.   , 0.088, 0.26 , 0.446],
       [0.   , 0.   , 0.081, 0.   , 0.088, 1.   , 0.097, 0.124],
       [0.184, 0.365, 0.324, 0.21 , 0.26 , 0.097, 1.   , 0.231],
       [0.086, 0.217, 0.104, 0.444, 0.446, 0.124, 0.231, 1.   ],
       [0.152, 0.   , 0.046, 0.199, 0.   , 0.   , 0.164, 0.146],
       [0.207, 0.141, 0.378, 0.076, 0.187, 0.131, 0.316, 0.208]])

# Item-Based CF

In [41]:
X_item = cosine_similarity(user_item_m.T)

In [42]:
X_item.shape

(479, 479)

In [43]:
X_item[:10,:11].round(2)

array([[1.  , 0.4 , 0.31, 0.31, 0.35, 0.49, 0.35, 0.49, 0.39, 0.19, 0.4 ],
       [0.4 , 1.  , 0.26, 0.49, 0.25, 0.42, 0.35, 0.35, 0.4 , 0.19, 0.36],
       [0.31, 0.26, 1.  , 0.18, 0.37, 0.24, 0.13, 0.25, 0.21, 0.16, 0.16],
       [0.31, 0.49, 0.18, 1.  , 0.21, 0.38, 0.41, 0.35, 0.35, 0.2 , 0.36],
       [0.35, 0.25, 0.37, 0.21, 1.  , 0.3 , 0.25, 0.35, 0.23, 0.2 , 0.21],
       [0.49, 0.42, 0.24, 0.38, 0.3 , 1.  , 0.35, 0.6 , 0.54, 0.18, 0.38],
       [0.35, 0.35, 0.13, 0.41, 0.25, 0.35, 1.  , 0.34, 0.29, 0.23, 0.4 ],
       [0.49, 0.35, 0.25, 0.35, 0.35, 0.6 , 0.34, 1.  , 0.58, 0.19, 0.35],
       [0.39, 0.4 , 0.21, 0.35, 0.23, 0.54, 0.29, 0.58, 1.  , 0.15, 0.31],
       [0.19, 0.19, 0.16, 0.2 , 0.2 , 0.18, 0.23, 0.19, 0.15, 1.  , 0.25]])

In [44]:
class CfRec():
    def __init__(self, M, X, items, k=20, top_n=10):
        self.X = X
        self.M = M
        self.k = k
        self.top_n = top_n
        self.items = items
        
    def recommend_user_based(self, user):
        ix = self.M.index.get_loc(user)
        # Use it to index the User similarity matrix
        u_sim = self.X[ix]
        # obtain the indices of the top k most similar users
        most_similar = self.M.index[u_sim.argpartition(-(self.k+1))[-(self.k+1):]]
        # Obtain the mean ratings of those users for all movies
        rec_items = self.M.loc[most_similar].mean(0).sort_values(ascending=False)
        # Discard already seen movies
        # already seen movies
        seen_mask = self.M.loc[user].gt(0)
        seen = seen_mask.index[seen_mask].tolist()
        rec_items = rec_items.drop(seen).head(self.top_n)
        # return recommendations - top similar users rated movies
        return (rec_items.index.to_frame()
                                .reset_index(drop=True)
                                .merge(self.items))

    def recommend_item_based(self, item):
        liked = self.items.loc[self.items.movieId.eq(item), 'title'].item()
        print(f"Because you liked {liked}, we'd recommend you to watch:")
        # get index of movie
        ix = self.M.columns.get_loc(item)
        # Use it to index the User similarity matrix
        i_sim = self.X[ix]
        # obtain the indices of the top k most similar users
        most_similar = self.M.columns[i_sim.argpartition(-(self.k+1))[-(self.k+1):]]
        return (most_similar.difference([item])
                                 .to_frame()
                                 .reset_index(drop=True)
                                 .merge(self.items)
                                 .head(self.top_n))

In [45]:
def because_user_liked(user_item_m, movies, ratings, user):
    ix_user_seen = user_item_m.loc[user]>0.
    seen_by_user = user_item_m.columns[ix_user_seen]
    return (seen_by_user.to_frame()
                 .reset_index(drop=True)
                 .merge(movies)
                 .assign(userId=user)
                 .merge(ratings[ratings.userId.eq(user)])
                 .sort_values('rating', ascending=False).head(10))

# Recommend User based

In [46]:
rec = CfRec(user_item_m, X_user, moviess)

In [47]:
because_user_liked(user_item_m, moviess, ratings, 69)

,movieId,title,genres,userId,rating,timestamp
34,1210,Star Wars: Episode VI - Return of the Jedi,Action|Adventure|Sci-Fi,69,5.0,1997-06-20 12:55:10
46,1291,Indiana Jones and the Last Crusade,Action|Adventure,69,5.0,1997-06-20 13:11:54
17,589,Terminator 2: Judgment Day,Action|Sci-Fi,69,5.0,1997-06-20 13:41:02
36,1222,Full Metal Jacket,Drama|War,69,5.0,1997-06-20 13:42:10
35,1213,Goodfellas,Crime|Drama,69,5.0,1997-06-20 13:42:10
29,1193,One Flew Over the Cuckoo's Nest,Drama,69,5.0,1997-06-20 14:22:59
30,1196,Star Wars: Episode V - The Empire Strikes Back,Action|Adventure|Sci-Fi,69,5.0,1997-06-20 13:41:02
32,1200,Aliens,Action|Adventure|Horror|Sci-Fi,69,4.0,1997-06-20 13:33:21
24,1073,Willy Wonka & the Chocolate Factory,Children|Comedy|Fantasy|Musical,69,4.0,1997-06-20 12:53:24
1,6,Heat,Action|Crime|Thriller,69,4.0,1997-06-20 12:53:24


In [48]:
rec.recommend_user_based(69)

,movieId,title,genres
0,527,Schindler's List,Drama|War
1,1214,Alien,Horror|Sci-Fi
2,541,Blade Runner,Action|Sci-Fi|Thriller
3,1225,Amadeus,Drama
4,1219,Psycho,Crime|Horror
5,924,2001: A Space Odyssey,Adventure|Drama|Sci-Fi
6,508,Philadelphia,Drama
7,1343,Cape Fear,Thriller
8,431,Carlito's Way,Crime|Drama
9,1292,Being There,Comedy|Drama


# Recommend Item based

In [49]:
rec = CfRec(user_item_m, X_item, moviess)

In [50]:
rec.recommend_item_based(1)

Because you liked Toy Story, we'd recommend you to watch:


,movieId,title,genres
0,110,Braveheart,Action|Drama|War
1,150,Apollo 13,Adventure|Drama|IMAX
2,296,Pulp Fiction,Comedy|Crime|Drama|Thriller
3,356,Forrest Gump,Comedy|Drama|Romance|War
4,480,Jurassic Park,Action|Adventure|Sci-Fi|Thriller
5,588,Aladdin,Adventure|Animation|Children|Comedy|Musical
6,589,Terminator 2: Judgment Day,Action|Sci-Fi
7,595,Beauty and the Beast,Animation|Children|Fantasy|Musical|Romance|IMAX
8,608,Fargo,Comedy|Crime|Drama|Thriller
9,1036,Die Hard,Action|Crime|Thriller


# CF model based using KNN

In [51]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [52]:
from fuzzywuzzy import fuzz

/Users/vibhavbagga/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [53]:
from scipy.sparse import csr_matrix
movie_user_mat_sparse = csr_matrix(user_item_m.T.values)

In [54]:
user_item_m.T

userId,1,2,3,4,5,6,7,8,9,10,...,7007,7008,7009,7010,7011,7012,7013,7014,7015,7016
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,0.0,0.0,5.0,0.0,4.0,0.0,4.0,...,5.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,3.0,5.0,2.0,0.0,0.0,0.0,...,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
# create mapper from movie title to index
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(moviess.set_index('movieId').loc[user_item_m.T.index].title))
}

In [56]:
movie_to_idx

{'Toy Story': 0,
 'Heat': 1,
 'Sabrina': 2,
 'Casino': 3,
 'Sense and Sensibility': 4,
 'Braveheart': 5,
 'Taxi Driver': 6,
 'Apollo 13': 7,
 'Die Hard: With a Vengeance': 8,
 'Before Sunrise': 9,
 'Clerks': 10,
 'Ed Wood': 11,
 'Little Women': 12,
 'Miracle on 34th Street': 13,
 'Once Were Warriors': 14,
 'Pulp Fiction': 15,
 "What's Eating Gilbert Grape": 16,
 'Forrest Gump': 17,
 "Carlito's Way": 18,
 'Dazed and Confused': 19,
 'In the Name of the Father': 20,
 'Jurassic Park': 21,
 'Naked': 22,
 'Philadelphia': 23,
 "Schindler's List": 24,
 'Short Cuts': 25,
 'Blade Runner': 26,
 'Tombstone': 27,
 'True Romance': 28,
 'Home Alone': 29,
 'Aladdin': 30,
 'Terminator 2: Judgment Day': 31,
 'Dances with Wolves': 32,
 'Beauty and the Beast': 33,
 'Fargo': 34,
 'Primal Fear': 35,
 'Underground': 36,
 'Dead Man': 37,
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb': 38,
 'Trainspotting': 39,
 "Singin' in the Rain": 40,
 "Breakfast at Tiffany's": 41,
 'Vertigo': 42,


In [57]:
%env JOBLIB_TEMP_FOLDER=/tmp
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)

env: JOBLIB_TEMP_FOLDER=/tmp


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [58]:
def fuzzy_matching(mapper, fav_movie, verbose=True):
    """
    return the closest match via fuzzy ratio. If no match found, return None
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True

    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]



def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    """
    return top n similar movie recommendations based on user's input movie


    Parameters
    ----------
    model_knn: sklearn model, knn model

    data: movie-user matrix

    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie

    n_recommendations: int, top n recommendations

    Return
    ------
    list of top n similar movie recommendations
    """
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input movie:', fav_movie)
    idx = fuzzy_matching(mapper, fav_movie, verbose=True)
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, reverse_mapper[idx], dist))

In [59]:
my_favorite = 'Toy Story'

make_recommendation(
    model_knn=model_knn,
    data=movie_user_mat_sparse,
    fav_movie=my_favorite,
    mapper=movie_to_idx,
    n_recommendations=10)

You have input movie: Toy Story
Found possible matches in our database: ['Toy Story', 'Toy Story 3', 'Toy Story 2', 'The Visitor']

Recommendation system start to make inference
......

Recommendations for Toy Story:
1: Terminator 2: Judgment Day, with distance of 0.49281641122114894
2: Pulp Fiction, with distance of 0.48155541016608483
3: Star Wars: Episode V - The Empire Strikes Back, with distance of 0.4781184365967812
4: Willy Wonka & the Chocolate Factory, with distance of 0.4699110846367115
5: Jurassic Park, with distance of 0.46704121675547483
6: Aladdin, with distance of 0.46607153801738843
7: Toy Story 2, with distance of 0.4613748712154705
8: Back to the Future, with distance of 0.4599876229334017
9: Forrest Gump, with distance of 0.45650760713505734
10: Star Wars: Episode VI - Return of the Jedi, with distance of 0.4471351181793153


# Matrix Factorization | SVD

In [60]:
user_item_m_n = user_item_m.to_numpy()
user_ratings_mean = np.mean(user_item_m_n, axis = 1)
R_demeaned = user_item_m_n - user_ratings_mean.reshape(-1, 1)

In [61]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [62]:
sigma = np.diag(sigma)

In [63]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = user_item_m.columns)

In [64]:
def recommend_movies(predictions_df, userID, moviess, ratings, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = ratings[ratings.userId == (userID)]
    user_full = (user_data.merge(moviess, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (moviess[~moviess['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

already_rated, predictions = recommend_movies(preds_df, 837, moviess, ratings, 10)

User 837 has already rated 21 movies.
Recommending the highest 10 predicted ratings movies not already rated.


In [65]:
already_rated.head(10)

,userId,movieId,rating,timestamp,title,genres
5,837,1358,5.0,2008-09-14 04:02:37,Sling Blade,Drama
20,837,48516,4.5,2008-09-14 03:59:03,"Departed, The",Crime|Drama|Thriller
15,837,5574,4.5,2008-09-14 03:54:19,"Transporter, The",Action|Crime
11,837,2959,4.5,2008-09-14 04:17:37,Fight Club,Action|Crime|Drama|Thriller
7,837,2146,4.5,2008-09-14 03:51:32,St. Elmo's Fire,Drama|Romance
16,837,7361,4.5,2008-09-14 04:17:37,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi
4,837,1290,4.5,2008-09-14 03:52:53,Some Kind of Wonderful,Drama|Romance
17,837,8807,4.0,2008-09-14 03:53:58,Harold and Kumar Go to White Castle,Adventure|Comedy
9,837,2859,4.0,2008-09-14 03:54:43,Stop Making Sense,Documentary|Musical
1,837,180,4.0,2008-09-14 03:46:41,Mallrats,Comedy|Romance


In [66]:
predictions

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1179,1210,Star Wars: Episode VI - Return of the Jedi,Action|Adventure|Sci-Fi
600,608,Fargo,Comedy|Crime|Drama|Thriller
1048,1073,Willy Wonka & the Chocolate Factory,Children|Comedy|Fantasy|Musical
5,6,Heat,Action|Crime|Thriller
1166,1196,Star Wars: Episode V - The Empire Strikes Back,Action|Adventure|Sci-Fi
619,628,Primal Fear,Crime|Drama|Mystery|Thriller
6,7,Sabrina,Comedy|Romance
1037,1061,Sleepers,Thriller
763,778,Trainspotting,Comedy|Crime|Drama


In [67]:
# Create an empty dataframe to store all predictions
all_predictions = pd.DataFrame(columns=['userId', 'movieId', 'title', 'Predictions'])

# Iterate over each user
for user_id in user_item_m.index:
    if user_id in preds_df.index:
        # Generate recommendations for the user
        _, user_predictions = recommend_movies(preds_df, user_id, moviess, ratings, 10)
        
        # Add user_id column to predictions
        user_predictions['userId'] = user_id
        
        # Append user predictions to the all_predictions dataframe
        all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)

# Save the predictions to a CSV file
predict = all_predictions.to_csv('all_predictions.csv', index=False)



User 1 has already rated 175 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 2 has already rated 61 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 3 has already rated 187 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 4 has already rated 28 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 5 has already rated 66 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 6 has already rated 24 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 7 has already rated 276 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 8 has already rated 70 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 9 has already rated 35 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 10 has already rated

/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)
/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)
/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user

User 14 has already rated 243 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 15 has already rated 49 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 16 has already rated 60 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 17 has already rated 26 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 18 has already rated 121 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 19 has already rated 50 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 20 has already rated 28 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 21 has already rated 163 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 22 has already rated 133 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 23 has alr

/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)
/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)
/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user

User 28 has already rated 32 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 29 has already rated 177 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 30 has already rated 31 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 31 has already rated 246 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 32 has already rated 73 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 33 has already rated 50 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 34 has already rated 93 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 35 has already rated 195 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 36 has already rated 20 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 37 has alre

...

/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)
/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)
/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user

User 6995 has already rated 259 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 6996 has already rated 121 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 6997 has already rated 33 movies.
Recommending the highest 10 predicted ratings movies not already rated.
User 6998 has already rated 33 movies.
Recommending the highest 10 predicted ratings movies not already rated.


/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)
/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)
/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user

User 6999 has already rated 20 movies.
Recommending the highest 10 predicted ratings movies not already rated.


/var/folders/vb/2jnsk8_x5ks907fbgv76ft9c0000gn/T/ipykernel_63685/254669969.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(user_predictions[['userId', 'movieId', 'title']].rename(columns={'Predictions': 'rating'}), ignore_index=True)
